# Exam Randomizer - Documented notebook

## About

Authors: Alexander BINDER, Matthieu DE MARI

Email: alexander_binder@sutd.edu.sg, matthieu_demari@sutd.edu.sg

## Description

Text

## About additional files

Text

## Necessary imports

Text

In [1]:
import numpy as np
import os
import subprocess
import math
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

### MikTeX (or any other pdflatex command line compiler)

Text

### PDF command line (Windows)

https://www.pdflabs.com/tools/pdftk-server/

## II. Preparing students IDs and passwords

### II.1. Retrieving students ID list from CSV file

In [2]:
def retrieve_student_list_IDs():
    
    # CSV file name
    studentidentifiers_file = './studentids.csv'
    
    # Initialize list of students ID as an empty list
    studentidentifiers = []
    
    # Open file as text and read each line
    with open(studentidentifiers_file,'r') as f:
        for line in f:
            # Read line and strip
            v = line.rstrip()
            
            # This is a cheap way to ignore things that are not student IDs
            if len(v) >= 2:
                # If it is an ID, append to list of IDs
                studentidentifiers.append(v)
                
    '''
    FOR DEBUGGING: Ignore.
    '''
    if len(studentidentifiers) == 0:
        print('len(studentidentifiers)==0')
        exit()
        
    return studentidentifiers

In [3]:
studentidentifiers = retrieve_student_list_IDs()
print(studentidentifiers)

['1002849', '1002945', '1002939', '1002948', '1002027', '1002675', '1002744', '1002811', '1002898', '1002871', '1002758', '1002870', '1002972', '1002949', '1000749', '1003056', '1002899', '1000593', '1002681', '1001781', '1002332', '1001834', '1002673', '1002992', '1002966', '1002656', '1002777', '1002125', '1002959', '1002954', '1002975', '1002923', '1002080', '1002686', '1003045', '1003107', '1003022', '1002974', '1002819', '1003044', '1002901', '1002722', '1002746', '1002887', '1002768', '1002968', '1002845', '1002653', '1003052', '1003101', '1002953', '1002963', '1002885', '1002965', '1002853', '1002029', '1002961', '1002973', '1002779', '1002872', '1002791']


### II.2. Generating passwords for students

In [4]:
def pseudopwgen():
    
    # String containing all usable characters for password generation
    chars = 'zxcvbnmasdfghjklqwertyuiop1234567890-?!@#$%^&*(='
    
    # Length of password
    pwlength = 12
    
    # Random generator for password
    # inds is a list of pwlength = 12 indexes, with values in [0, len(chars)]
    inds = np.random.choice(len(chars), size = pwlength, replace = True, p = None)
    
    # Initialize password as an empty string
    pseudopw = ''
    
    # Add randomly selected characters from chars to pseudopw
    for k in range(pwlength):
        pseudopw += chars[inds[k]]
        
    return pseudopw

In [5]:
generated_password = pseudopwgen()
print(generated_password)

7iuyzq=@zu#6


### II.3. Matching passwords and IDs

In [6]:
def generate_ID_password():
    
    # Retrieve list of students IDs
    studentidentifiers = retrieve_student_list_IDs()
    number_students = len(studentidentifiers)
    
    # Generate passwords
    studentpws = [pseudopwgen() for _ in range(number_students)]
    
    # Dictionary of matching values
    id_password_dict = {studentidentifiers[i]: studentpws[i] for i in range(number_students)}
    
    return id_password_dict

In [7]:
id_password_dict = generate_ID_password()
print(id_password_dict)

{'1002849': '!wr1l6vpsm2m', '1002945': '1611fh-t1v*j', '1002939': 'c=-0@%^0q5x#', '1002948': 'z=gh$1c*0rh!', '1002027': '0j^&ywf2e*r^', '1002675': 'gu9oadu=219?', '1002744': 'j!n?s3wydsk0', '1002811': 'x^teftd#4ijn', '1002898': 'wzjt?5%maew3', '1002871': 'mow-*!g22d53', '1002758': 'm5mx3m9$fmrt', '1002870': '-u@d3qdbsa(h', '1002972': 'io31sbu4w0lr', '1002949': '6i8k8j9jmw==', '1000749': '@=19ey*fdxki', '1003056': 'ccdqw3h^&l4a', '1002899': '1y*l7pi*^1#x', '1000593': 'yhihm44^-ko4', '1002681': 'x=^wn74s%f02', '1001781': '$%kt225h#yo#', '1002332': '9jxvw^h!r6?0', '1001834': 'hah8qoe5kb00', '1002673': 'y*jk(hw&m?rc', '1002992': '(3k3yy-a3^1l', '1002966': 'shhpfebgme@5', '1002656': 'ddx97kk7wbd#', '1002777': 'kyoy!drifwb^', '1002125': 'zv#a2fif#1&i', '1002959': '8!i*cigp!aqs', '1002954': '0q6*8@awfezk', '1002975': '5^zy*o*drh=r', '1002923': '5=&d1ps*g6ru', '1002080': 'pwyhba6l(9$#', '1002686': 'l0q7p(pm*$hp', '1003045': 'fx(io(t&s@85', '1003107': '70&bxy8z@k72', '1003022': 'mccp?=^#0@$9', 

### II.4. Save passwords and IDs to file

In [8]:
def save_ID_password(id_password_dict):
    
    with open('studentpasswords.txt', 'w') as f:
        for key, value in id_password_dict.items():
            f.write("{} - {}\n".format(key, value))
        f.close()

In [9]:
save_ID_password(id_password_dict)

## III. Randomizing values on tasks

### III.1. Randomizing values for a given task

In [10]:
class solutionscrubber_t1a():
    
    def __init__(self, studentid_as_seed):
        
        # Use the student ID as seed
        # Remeber to convert student ID from str to int!
        self.studentid_as_string = studentid_as_seed
        self.studentid_as_seed = int(self.studentid_as_string)
        np.random.seed(self.studentid_as_seed)
        
        # Scrub keys to look for in LaTeX file
        # '#ra01' = img size 1, 
        # '#ra02' = img size 2,
        # '#ra03' = kernel size 1
        # '#ra04' = kernel size 2
        # '#ra05' = stride
        # '#ra06' = padding size
        self.scrubkeys = ['#ra01', '#ra02', '#ra03', '#ra04', '#ra05', '#ra06']
        
        # Dict of all acceptable random values
        self.random_values_dict_all = {'#ra01': [32, 64, 128, 256, 512],
                                       '#ra02': [32, 64, 128, 256, 512],
                                       '#ra03': [3, 5, 7],
                                       '#ra04': [3, 5, 7],
                                       '#ra05': [1, 2, 3],
                                       '#ra06': [1, 2, 3]}
        
        # Dict of values for given student
        # Initialized as None for now
        self.random_values_dict_student = {key: None for key in self.scrubkeys}
        
        # Randomize values
        self.randomize_values()
        
        # Dict of values for given student (solutions)
        # '#ra07' = output size 1
        # '#ra08' = output size 2
        self.dict_student_solutions = {'#ra07': None, '#ra08': None}
        
        # Compute solutions
        self.compute_solution()
        
        
    def randomize_values(self):
        
        for key in self.random_values_dict_student.keys():
            # Get a random val for each key among the possible choices given in self.random_values_dict_all
            random_val = np.random.choice(self.random_values_dict_all[key])
            
            # Assign it to self.random_values_dict_student
            self.random_values_dict_student[key] = random_val
            

    def compute_solution(self):
        
        # Retrieve parameters: image size
        imgsize_h = self.random_values_dict_student['#ra01']
        imgsize_w = self.random_values_dict_student['#ra02']
        
        # Retrieve parameters: kernel size
        ksize_h = self.random_values_dict_student['#ra03']
        ksize_w = self.random_values_dict_student['#ra04']
        
        # Retrieve parameters: stride size
        stride = self.random_values_dict_student['#ra05']
        
        # Retrieve parameters: padding size
        pad =  self.random_values_dict_student['#ra06']
        
        # Compute solutions
        self.dict_student_solutions['#ra07'] = math.floor((imgsize_h - ksize_h + 2*pad)/float(stride) + 1)
        self.dict_student_solutions['#ra08'] = math.floor((imgsize_w - ksize_w + 2*pad)/float(stride) + 1)

In [11]:
student_id = '0002403'
scrubber_for_task1a_given_student = solutionscrubber_t1a(studentid_as_seed = student_id)
#print(scrubber_for_task1a_given_student.__dict__, end = '\n\n')
print(scrubber_for_task1a_given_student.random_values_dict_all, end = '\n\n')
print(scrubber_for_task1a_given_student.random_values_dict_student, end = '\n\n')
print(scrubber_for_task1a_given_student.dict_student_solutions, end = '\n\n')

{'#ra01': [32, 64, 128, 256, 512], '#ra02': [32, 64, 128, 256, 512], '#ra03': [3, 5, 7], '#ra04': [3, 5, 7], '#ra05': [1, 2, 3], '#ra06': [1, 2, 3]}

{'#ra01': 256, '#ra02': 32, '#ra03': 7, '#ra04': 3, '#ra05': 3, '#ra06': 2}

{'#ra07': 85, '#ra08': 12}



### III.2. Inputing into LaTeX file

In [12]:
def input_randomized_values_into_latex(task_scrubber_object):
    
    # Make a copy of the original template
    student_id = task_scrubber_object.studentid_as_string
    
    # Retrieve LaTeX code as string from file
    path_to_original_file = './latex_file_for_demo.tex'
    with open(path_to_original_file, 'r') as f:
        latex_code_as_string = f.read()
        f.close()
        
    # Replace randomized values in LaTeX string
    for key, val in task_scrubber_object.random_values_dict_student.items():
        latex_code_as_string = latex_code_as_string.replace(str(key), str(val))
    
    # Replace solutions in LaTeX string
    for key, val in task_scrubber_object.dict_student_solutions.items():
        latex_code_as_string = latex_code_as_string.replace(str(key), str(val))
    
    # Check for existing directory, otherwise create it
    sub_dir = './students_pdfs/'
    if not os.path.exists(sub_dir):
        os.mkdir(sub_dir)
    stud_dir = sub_dir + '{}/'.format(student_id)
    if not os.path.exists(stud_dir):
        os.mkdir(stud_dir)
    
    # Rewrite to file (student copy)
    path_to_new_file = stud_dir + 'latex_file_for_demo_{}.tex'.format(student_id)
    with open(path_to_new_file, 'w') as f:
        f.write(latex_code_as_string)
        f.close()
        
    # Make master copy with solutions
    latex_code_as_string = latex_code_as_string.replace('noanswers', 'answers')
    
    # Rewrite to file (solutions copy)
    path_to_new_file = stud_dir + 'latex_file_for_demo_{}_solutions.tex'.format(student_id)
    with open(path_to_new_file, 'w') as f:
        f.write(latex_code_as_string)
        f.close()

In [13]:
input_randomized_values_into_latex(task_scrubber_object = scrubber_for_task1a_given_student)

## III.3. Compile LaTeX file from python

In [14]:
def compile_for_student_windows(student_id, remove = False):
    
    # Compile LaTeX file (student copy)
    sub_dir = './students_pdfs/'
    stud_dir = sub_dir + '{}/'.format(student_id)
    path_to_new_file = stud_dir + 'latex_file_for_demo_{}.tex'.format(student_id)
    os.system("pdflatex -halt-on-error -output-directory {} {}".format(stud_dir, path_to_new_file))
    
    # Compile LaTeX file (solutions copy)
    sub_dir = './students_pdfs/'
    stud_dir = sub_dir + '{}/'.format(student_id)
    path_to_new_file = stud_dir + 'latex_file_for_demo_{}_solutions.tex'.format(student_id)
    os.system("pdflatex -halt-on-error -output-directory {} {}".format(stud_dir, path_to_new_file))
    
    # Remove extra files (just leave the PDF)
    if remove:
        file = stud_dir + 'latex_file_for_demo_{}.aux'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}.log'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}.tex'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}.out'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}_solutions.aux'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}_solutions.log'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}_solutions.tex'.format(student_id)
        if os.path.exists(file):
            os.remove(file)
        file = stud_dir + 'latex_file_for_demo_{}_solutions.out'.format(student_id)
        if os.path.exists(file):
            os.remove(file)

In [15]:
compile_for_student_windows(student_id)

### III.4. Add password to PDF

In [16]:
def protect_pdf(student_id, password, dummy_password = 'chicken'):
    
    # Protect PDF with password
    sub_dir = './students_pdfs/'
    stud_dir = sub_dir + '{}/'.format(student_id)
    path_to_old_pdf = stud_dir + 'latex_file_for_demo_{}.pdf'.format(student_id)
    path_to_new_pdf = stud_dir + 'latex_file_for_demo_{}_pw.pdf'.format(student_id)
    query = "pdftk {} output {} owner_pw {} user_pw {}".format(path_to_old_pdf, path_to_new_pdf, \
                                                               password, dummy_password)
    # One of these should work!
    os.system(query)
    subprocess.run(query, shell = True)
    subprocess.call(query, shell = True)
    
    # Remove unprotected PDF
    if os.path.exists(path_to_old_pdf):
        os.remove(path_to_old_pdf)

In [17]:
password = 'hello1234'
protect_pdf(student_id, password)

## IV. Sending via email, with Python

### IV.1 Retreive mail credentials and setup mailer

Text

In [18]:
def setup_mailer():
    
    gmail_user = 'matthieu.de.mari@gmail.com'
    gmail_password = 'P@ssword!'
    
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
    except:
        print("Something is wrong with your email credentials. Check setup_mailer() function.")
    return server

In [19]:
mailer = setup_mailer()
print(mailer.__dict__)

Something is wrong with your email credentials. Check setup_mailer() function.
{'keyfile': None, 'certfile': None, 'context': <ssl.SSLContext object at 0x000001982BE80C18>, '_host': 'smtp.gmail.com', 'timeout': <object object at 0x00000198102FC720>, 'esmtp_features': {'size': '35882577', '8bitmime': '', 'auth': ' LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH', 'enhancedstatuscodes': '', 'pipelining': '', 'chunking': '', 'smtputf8': ''}, 'command_encoding': 'ascii', 'source_address': None, 'sock': <ssl.SSLSocket fd=1296, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.1.129', 55610), raddr=('74.125.200.108', 465)>, 'file': <_io.BufferedReader name=1296>, 'local_hostname': '[192.168.1.129]', 'ehlo_resp': b'smtp.gmail.com at your service, [27.104.213.86]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8', 'does_esmtp': 1, 'user': 'matthieu.de.mari@gmail

### IV.2. Send PDF and password to student via mail

Text

In [20]:
def send_pdf_to_student(mailer, student_id, password, target_email):
    
    # 1. Prepare email
    # - Body as string
    body_str = 'Hello there!\n'
    body_str += 'This is an automated email, for student {}, with a password protected PDF!\n'.format(student_id)
    body_str += 'To unlock your PDF, use the following password: {}.\n'.format(password)
    body_str += 'Good luck on your exam!\n'
    body_str += 'Best regards,\n'
    body_str += 'Matthieu\n'
    # - Create instance of MIMEMultipart 
    msg = MIMEMultipart() 
    # - Sender   
    msg['From'] = mailer.user
    # - Recipient 
    msg['To'] = target_email 
    # - Subject  
    msg['Subject'] = 'Secret exam with password protected PDF attached and password in the body of this email!'
    # - Body
    msg.attach(MIMEText(body_str, 'plain'))
    
    # 2. Retrieve PDF file and attach to mail
    # - Retrieve PDF
    sub_dir = './students_pdfs/'
    stud_dir = sub_dir + '{}/'.format(student_id)
    path_to_pdf = stud_dir + 'latex_file_for_demo_{}_pw.pdf'.format(student_id)
    f = open(path_to_pdf, 'rb')
    # - Initialize instance of MIMEBase and named as p 
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((f).read()) 
    # - Encode into base64
    encoders.encode_base64(p)
    # - Attach to mail.
    p.add_header('Content-Disposition', "attachment; filename= %s" % path_to_pdf)
    msg.attach(p)
    
    # 3. Send mail
    try:
        mailer.login(mailer.user, mailer.password)
        mailer.sendmail(mailer.user, target_email, msg.as_string())
        mailer.close()
        status = True
    except:
        print("Something is wrong with your email credentials. Check send_pdf_to_student() function.")
        status = False
    return status

In [21]:
target_email = 'matthieu.de.mari@gmail.com'
status = send_pdf_to_student(mailer, student_id, password, target_email)

Something is wrong with your email credentials. Check send_pdf_to_student() function.


## V. Run all!

In [22]:
def run_all(make_it_official = False):
    
    # Display
    print("Starting randomized exam generation...")
    
    # 1. Generate ID-password combinations
    id_password_dict = generate_ID_password()
    save_ID_password(id_password_dict)
    
    # Display
    print("---")
    print("1. Generated ID-password combinations. Displaying below.")
    print(id_password_dict)
    print("---")
    
    # For each student
    for student_id, password in id_password_dict.items():
        # Display
        print("-")
        print("Exam generation for student {}.".format(student_id))
        
        # 2.a Generate randomized task 1 values (repeat for all tasks)
        scrubber_for_task1a_given_student = solutionscrubber_t1a(studentid_as_seed = student_id)
        
        # 2.b Insert value for task 1 into LaTeX (repeat for all tasks)
        input_randomized_values_into_latex(task_scrubber_object = scrubber_for_task1a_given_student)
        
        # 2.c. Compile LaTeX
        compile_for_student_windows(student_id, remove = True)
        
        # 2.d. Password protection
        protect_pdf(student_id, password)
        
        # Display
        print("Exam generation complete.")
        
    # Display
    print("---")
    print("2. Generated all encrypted PDFs.")
    
    # Setup mailer
    mailer = setup_mailer()
        
    # For each student
    for student_id, password in id_password_dict.items():
        print("-")
        print("Sending email to student {}.".format(student_id))
        
        # 3. Send email to student
        if not make_it_official:
            target_email = 'matthieu.de.mari@gmail.com'
        else:
            target_email = 'matthieu.de.mari@gmail.com'
            #target_email = '{}@mymail.sutd.edu.sg'.format(student_id)
        status = send_pdf_to_student(mailer, student_id, password, target_email)
        if not status:
            assert False, "There is an error in your mailer credentials." \
                            + "Please investigate your send_pdf_to_student() fuction."
        
        # Display
        print("Email to student {} has been sent.".format(student_id))

    # Display
    print("3. Emails have been sent to students.")
    print("---")
    print("Task completed.")

In [23]:
#run_all()